In [1]:
import datetime as dt

import pandas as pd
from keras import Sequential
from keras.losses import categorical_crossentropy
from lazyft.data_loader import load_pair_data

from time_series_to_gaf.constants import TRAIN_IMAGES_PATH, REPO
import keras
from keras.initializers import initializers_v2 as initializer

from time_series_to_gaf.preprocess import data_to_image_preprocess
from time_series_to_gaf.constants import TEST_IMAGES_PATH
from time_series_to_gaf.cnn_model import create_cnn
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizer_v2.adam import Adam
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
pair = 'BTC/USDT'
SPLIT = 0.30
LR = 0.001
TIMESTAMP = dt.datetime.now().strftime("%Y%m%d%H%M%S")
# cnn_networks = 1
models: list[keras.Sequential] = []
image_train_path = TRAIN_IMAGES_PATH / pair.replace('/', '_')
image_test_path = TEST_IMAGES_PATH / pair.replace('/', '_')
image_train_path.mkdir(parents=True, exist_ok=True)
image_test_path.mkdir(parents=True, exist_ok=True)
data_to_image_preprocess(
    timerange='20170101-20211231', image_save_path=image_train_path, pair=pair, interval='1h'
)
data_to_image_preprocess(timerange='20220101-', pair=pair, image_save_path=image_test_path)

# global history, string_list, summary
target_size = 40
batch_size = 20
EPOCHS = 10

In [7]:


initializers_ = [
    initializer.Orthogonal(),
    initializer.LecunUniform(),
    initializer.VarianceScaling(),
    initializer.RandomNormal(),
    initializer.RandomUniform(),
    initializer.TruncatedNormal(),
    initializer.GlorotNormal(),
    initializer.GlorotUniform(),
    initializer.HeNormal(),
    initializer.HeUniform(),
    initializer.Orthogonal(seed=42),
    initializer.LecunUniform(seed=42),
    initializer.VarianceScaling(seed=42),
    initializer.RandomNormal(seed=42),
    initializer.RandomUniform(seed=42),
    initializer.TruncatedNormal(seed=42),
    initializer.GlorotNormal(seed=42),
    initializer.GlorotUniform(seed=42),
    initializer.HeNormal(seed=42),
    initializer.HeUniform(seed=42),
]
# for i, initializer_ in enumerate(initializers_):
#     cnn = create_cnn(target_size, kernel_initializer=initializer_)
#     # Compile each model
#     cnn.compile(optimizer=Adam(learning_rate=LR), loss='binary_crossentropy', metrics=['acc'])
#     models.append(cnn)

# All images will be rescaled by 1./255
train_validate_datagen = ImageDataGenerator(
    rescale=1 / 255, validation_split=SPLIT
)  # set validation split
test_datagen = ImageDataGenerator(rescale=1 / 255)
# data_chunks = ensemble_data(len(models), str(image_path))
test_generator = test_datagen.flow_from_directory(
    directory=image_test_path,
    target_size=(target_size, target_size),
    batch_size=batch_size,
)

2022-04-03 09:45:23.250 | INFO     | lazyft.downloader:download_missing_historical_data:301 - Checking if download is needed for BTC/USDT @ 1h interval(s)
2022-04-03 09:45:23.253 | INFO     | lazyft.downloader:download_missing_historical_data:338 - Data is up to date
2022-04-03 09:45:23.315 | INFO     | lazyft.data_loader:load_pair_data:39 - Loaded 35041 rows for BTC/USDT, data starts at 2018-01-01 00:00:00+00:00
Generating images...
========PREPROCESS REPORT========:
Total Data Points: 1461
Total Images Created: 1441
Total LONG positions: 700
Total SHORT positions: 741
2022-04-03 09:45:32.889 | INFO     | lazyft.downloader:download_missing_historical_data:301 - Checking if download is needed for BTC/USDT @ 1h interval(s)
2022-04-03 09:45:32.891 | INFO     | lazyft.downloader:download_missing_historical_data:338 - Data is up to date
2022-04-03 09:45:32.940 | INFO     | lazyft.data_loader:load_pair_data:39 - Loaded 2209 rows for BTC/USDT, data starts at 2022-01-01 00:00:00+00:00
Generat

In [3]:
def load_model(pair: str, time: str, model_name: str) -> Sequential:
    model = create_cnn(40)
    model_to_load = REPO / pair.replace('/', '_') / time / 'models' / model_name
    model.load_weights(model_to_load)

    return model
model = load_model(pair, '20220403092229', 'HeUniform.h5')
model.compile(
            optimizer=Adam(learning_rate=LR), loss=categorical_crossentropy, metrics=['acc']
        )

In [11]:
scores = model.evaluate(test_generator)
print("Test {0}s: {1:.2f}%".format(model.metrics_names[1], scores[1] * 100))
string_list = []
model.summary(print_fn=lambda x: string_list.append(x))
string_list.append(f'test acc: {scores[1] * 100}')


1/4 [======>.......................] - ETA: 0s - loss: 0.6396 - acc: 0.7500

/home/raphael/PycharmProjects/freqgym/venv/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 [==============================] - 0s 109ms/step - loss: 0.6582 - acc: 0.6712
Test accs: 67.12%


In [13]:
scores

[0.6582338213920593, 0.6712328791618347]

In [15]:
sample = test_generator.next()

In [16]:
test_generator.class_indices


{'LONG': 0, 'SHORT': 1}

In [3]:
from datetime import timedelta
import time

btc = load_pair_data('BTC/USDT', timeframe='5m', timerange='20220101-')

# t1 = time.perf_counter()
# sampled = btc.resample('4H').mean()
# print('Elapsed time:', timedelta(seconds=time.perf_counter() - t1))
t1 = time.perf_counter()
gb = btc.groupby(pd.Grouper(key='date', freq='30min')).mean().reset_index()
print('Elapsed time:', timedelta(seconds=time.perf_counter() - t1))
# print(sampled.head())
print(gb.head())


2022-04-04 17:35:07.414 | INFO     | lazyft.downloader:download_missing_historical_data:301 - Checking if download is needed for BTC/USDT @ 5m interval(s)
2022-04-04 17:35:07.419 | INFO     | lazyft.downloader:download:370 - Downloading 93 days worth of market data for BTC/USDT @ 5m ticker-interval(s)...
2022-04-04 17:35:29.632 | INFO     | lazyft.downloader:download_watcher:430 - Downloaded history for BTC/USDT @ 5m (1/1)
2022-04-04 17:35:29.634 | INFO     | lazyft.downloader:download_missing_historical_data:333 - Finished downloading data for 1 pairs @ 5m
2022-04-04 17:35:29.634 | INFO     | lazyft.downloader:download_missing_historical_data:338 - Data is up to date
2022-04-04 17:35:29.900 | INFO     | lazyft.data_loader:load_pair_data:39 - Loaded 27042 rows for BTC/USDT, data starts at 2022-01-01 00:00:00+00:00
Elapsed time: 0:00:00.003820
                       date  open  high   low  close  volume
0 2022-01-01 00:00:00+00:00 46.3K 46.4K 46.3K  46.3K 117.776
1 2022-01-01 00:30:00+0